In [ ]:
# Libs
import json
import csv
import pandas as pd
import numpy as np
import time
import sys
import random
import matplotlib.pyplot as plt
from joblib import dump, load
from glob import glob
from joblib import dump, load

from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split

In [ ]:
# Load the african country dictionary
dict_path = "data/output/african_countries.json"

with open(dict_path) as json_file:
    ccDict = json.load(json_file)

In [ ]:
# Load all features
with open('data/output/mil_exp.json') as json_file:
    mil_exp = json.load(json_file)
    
with open('data/output/population.json') as json_file:
    population = json.load(json_file)
    
with open('data/output/arms_imports.json') as json_file:
    arms_imports = json.load(json_file)
    
with open('data/output/conflicts.json') as json_file:
    conflicts = json.load(json_file)
    
with open('data/output/mil_pers.json') as json_file:
    mil_pers = json.load(json_file)
    

In [ ]:
# Get the time windows we will work with
minYear = 1962
maxYear = 2012

# Go through all datasets
temp_df = []
for i, key in enumerate(ccDict):
    
    # Get the country name
    country_name = ccDict[key]['name']
    
    # Go through years
    for year in range(minYear, maxYear+1):
    
        # datum
        datum = {
            'COW_key': int(key),
            'Year': int(year),
            'Mil_Exp':float(mil_exp[key][str(year)]),
            'Population':int(population[key][str(year)]),
            'Mil_Pers':float(mil_pers[key][str(year)]),
            'Arms_Imports':int(arms_imports[key][str(year)]),
            'Conflict':conflicts[key][str(year)]
        }
        
        # Append to temp df
        temp_df.append(datum)

# Convert temp df to pandas
df = pd.DataFrame(temp_df) 
    

# Print nbr of rows
print("Nbr of rows : " + str(len(df.index)))

# Preview df
df.head(10)

## Missing Data

In [ ]:
df.mean(axis = 0)

In [ ]:
mean_milexp = df.mean(axis = 0)['Mil_Exp']
mean_milpers = df.mean(axis = 0)['Mil_Pers']


for index, row in df.iterrows():
    
    if(row['Mil_Exp'] == 0):
        df.at[index,'Mil_Exp'] = mean_milexp
    
    if(row['Mil_Pers'] == 0):
        df.at[index,'Mil_Pers'] = mean_milpers

## Balance Dataset

In [ ]:
# Shuffle Rows
df = df.sample(frac=1).reset_index(drop=True)

# count excess
imbalance = df['Conflict'].value_counts()

excessLabel = 0
if(imbalance[0] > imbalance[1]):
    excessLabel = 0
else:
    excessLabel = 1

# Nbr of excess
diff = abs(imbalance[0] - imbalance[1])

In [ ]:
balanced_df = df.copy()

nbr_dropped = 0
for index, row in balanced_df.iterrows():
    
    if(nbr_dropped >= diff):
        break
    
    if(row['Conflict'] == excessLabel):
        balanced_df.drop(index, inplace=True)
        nbr_dropped += 1

In [ ]:
balanced_df['Conflict'].value_counts()

## Split Dataset

In [ ]:
# Split features/label
features = ['Arms_Imports', 'Mil_Exp', 'Mil_Pers', 'Population', 'Year']
label = ['Conflict']
X = balanced_df[features]
y = balanced_df[label]

In [ ]:
# Split the data
train_X, valid_X, train_Y, valid_Y = train_test_split(X, y, test_size=0.01, random_state=12, shuffle=True, stratify=y)

# cast to np
valid_Y = np.array(valid_Y)
valid_X = np.array(valid_X)

print("Length of training set : ", len(train_X))
print("Length of validation set : ", len(valid_X))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rdf_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
rdf_classifier.fit(train_X,train_Y)

In [ ]:
rdf_predictions = rdf_classifier.predict(valid_X)

In [ ]:
success = 0
for i, pred in enumerate(rdf_predictions):
    if(pred == valid_Y[i]):
        success += 1
        
print("Validation Accuracy = " + str(success/len(train_Y)))

## Logistic Regression

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize data
scaler = MinMaxScaler()
train_X_t_n = scaler.fit_transform(train_X)
valid_X_t_n = scaler.transform(valid_X)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_classifier = LogisticRegression()
log_classifier.fit(train_X_t_n, train_Y)

In [ ]:
log_predictions = log_classifier.predict(valid_X_t_n)

In [ ]:
success = 0
for i, pred in enumerate(log_predictions):
    if(pred == valid_Y[i]):
        success += 1
        
print("Validation Accuracy = " + str(success/len(train_Y)))

## SVM Algorithms

In [ ]:
from sklearn import svm

svm_classifier = svm.SVC(gamma='auto',probability=True)
svm_classifier.fit(train_X_t_n, train_Y)

In [ ]:
svm_predictions = svm_classifier.predict(valid_X_t_n)

In [ ]:
success = 0
for i, pred in enumerate(svm_predictions):
    if(pred == valid_Y[i]):
        success += 1
        
print("Validation Accuracy = " + str(success/len(train_Y)))

## Save Model

In [ ]:
dump(rdf_classifier, 'data/model/model.joblib') 

## Dataviz Output

In [ ]:
print(features)

In [ ]:
# Init a dict that will contain the total value of arms import per year per country
predict_dict = {}

nbrOfKey = len(ccDict.keys())

for i, key in tqdm(enumerate(ccDict), total=nbrOfKey):
    predict_dict[key] = {}
    
    for year in range(minYear, maxYear+1):
        
        # create datum in SAME order
        datum = [arms_imports[key][str(year)], float(mil_exp[key][str(year)]), float(mil_pers[key][str(year)]),int(population[key][str(year)]),year]
        
        # predict with probabibility
        predict_dict[key][str(year)] = str(rdf_classifier.predict_proba([datum])[0][1])

In [ ]:
# Save the dict to a json file
with open('data/output/predictions.json', 'w') as fp:
    json.dump(predict_dict, fp)